メモ
I91,95,84あたりは多い

<table>
  <thead>
  <tr>
    <td>更新日時</td>
    <td>変更内容</td>    
  </tr>
  </thead>
  <tr>
    <td>0613 0930</td>
    <td>dayを追加。officerを除去</td> 
  </tr>
  <tr>
    <td>0613 1000</td>
    <td>dayを追加。location/officerは追加</td> 
  </tr>
</table>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
df = pd.read_csv("train.csv")

/home/minoda/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df[df["driver_race"]=="Other"]["driver_race_raw"].value_counts()

Native American    1220
Name: driver_race_raw, dtype: int64

以下、データの成形を行なっていく。

Violationに関しては、カンマ区切りにされているviolationの種類をstr.splitで分割した上で、全てのviolationの種類をまとめたリストを作成する。このリストをカラム名とし、各indexについて対応するviolationの値を1とする。

In [56]:
# violationの種類を検索
df_violation = df["violation"].str.split(',',expand=True)
violation_list = np.array([])

"""
for i in range(0,4):
    violation_list = np.append(violation_list,df_violation[0].unique())
violation_list = np.unique(violation_list)
"""

violation_list = np.array(['Cell phone', 'Equipment', 'License', 'Lights', 'Moving violation',
       'Other', 'Registration/plates', 'Safe movement', 'Seat belt',
       'Speeding', 'Stop sign/light'], dtype=object)

for list in violation_list:
    df["violation_"+list] = np.zeros(df.contraband_found.count())
    for i in range(0,4):
        df.loc[df_violation[i]==list,"violation_"+list]=1

In [57]:
# stop_durationを数値に直す
# 2回目以降の実行ではエラー
df.loc[df["stop_duration"]=="1-15 min","stop_duration"]=0
df.loc[df["stop_duration"]=="16-30 min","stop_duration"]=1
df.loc[df["stop_duration"]=="30+ min","stop_duration"]=2

# driver_genderを数値に直す
# 2回目以降の実行ではエラー
df.loc[df["driver_gender"]=="M","driver_gender"]=0
df.loc[df["driver_gender"]=="F","driver_gender"]=1

In [58]:
df["driver_race"].value_counts()

White       71564
Black       11067
Hispanic     9126
Asian        1731
Other         506
Name: driver_race, dtype: int64

In [59]:
df['stop_time'] = pd.to_datetime(df['stop_time'])
df['stop_date'] = pd.to_datetime(df['stop_date'])
df["stop_weekday_name"] = df["stop_date"].dt.weekday_name
df["stop_month"] = df["stop_date"].dt.month    
df["stop_year"] = df["stop_date"].dt.year
df["stop_hour"] = df["stop_time"].dt.hour
df["stop_day"] = df["stop_date"].dt.day

In [60]:
df = pd.get_dummies(df,columns=["stop_month"], prefix='stop_month', prefix_sep='_')
df = pd.get_dummies(df,columns=["stop_year"], prefix='stop_year', prefix_sep='_')
df = pd.get_dummies(df,columns=["stop_weekday_name"], prefix='stop_day', prefix_sep='_')
df = pd.get_dummies(df,columns=["stop_hour"], prefix='stop_hour', prefix_sep='_')

df = pd.get_dummies(df,columns=["driver_race"],prefix='race',prefix_sep='_')

df = pd.get_dummies(df,columns=["search_type"], prefix='search', prefix_sep='_')

df = pd.get_dummies(df,columns=["county_name"], drop_first=True, prefix='county', prefix_sep='_')

In [61]:
df = df.drop(["stop_date","stop_time","search_type_raw","search_conducted","county_fips"
              ,"driver_race_raw","violation","violation_raw","state"
              ,"police_department","driver_age"],axis=1)

In [62]:
df["contraband_found"] = df["contraband_found"]*1

In [63]:
#df["location_raw"] = df["location_raw"].fillna("N/A")
df["fine_grained_location"] = df["fine_grained_location"].fillna("N/A")

In [64]:
df["location_exit"] = 0
df.loc[df["fine_grained_location"].str.contains("X"),"location_exit"] = 1
df.loc[df["fine_grained_location"].str.contains("x"),"location_exit"] = 1
df.loc[df["fine_grained_location"].str.contains("sex"),"location_exit"] = 0

df["location_route"] = 0
df.loc[df["fine_grained_location"].str.contains("rt"),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("rte "),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("RT."),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("RT "),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("rT "),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("Route"),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("Rte"),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("Rt"),"location_route"] = 1

df["location_00000"] = 0
df.loc[df["fine_grained_location"].str.contains("000"),"location_00000"] = 1

In [65]:
df["location_raw"].unique()
location_list = np.array(['berlin', 'greenwich', 'brooklyn', 'essex', 'union', 'shelton',
       'naugatuck', 'manchester', 'meriden', 'bridgeport', 'chester',
       'montville', 'oxford', 'beacon falls', 'waterford', 'ledyard',
       'middlefield', 'southbury', 'killingly', 'andover', 'colchester',
       'trumbull', 'bozrah', 'windsor', 'westport', 'litchfield',
       'coventry', 'plainfield', 'thompson', 'stonington', 'stafford',
       'middletown', 'harwinton', 'washington', 'stratford', 'columbia',
       'new haven', 'mansfield', 'griswold', 'windham', 'hebron',
       'old saybrook', 'west haven', 'torrington', 'preston', 'sterling',
       'north stonington', 'vernon', 'deep river', 'hartford',
       'brookfield', 'darien', 'enfield', 'new britain', 'old lyme',
       'norwalk', 'willington', 'branford', 'wethersfield', 'sherman',
       'marlborough', 'somers', 'hamden', 'waterbury', 'barkhamsted',
       'stamford', 'clinton', 'ashford', 'east haddam', 'danbury',
       'lebanon', 'southington', 'watertown', 'milford', 'tolland',
       'windsor locks', 'madison', 'orange', 'westbrook', 'putnam',
       'north haven', 'ellington', 'burlington', 'franklin',
       'bridgewater', 'groton', 'fairfield', 'new hartford', 'bolton',
       'guilford', 'wallingford', 'newtown', 'east hartford', 'granby',
       'woodbridge', 'farmington', 'new canaan', 'norwich', 'voluntown',
       'sharon', 'durham', 'haddam', 'rocky hill', 'salem', 'bethlehem',
       'seymour', 'winchester', 'thomaston', 'lisbon', 'roxbury',
       'cromwell', 'east granby', 'west hartford', 'woodbury',
       'middlebury', 'cornwall', 'east windsor', 'north canaan',
       'cheshire', 'norfolk', 'east lyme', 'derby', 'newington',
       'woodstock', 'goshen', 'new milford', 'bethany', 'scotland',
       'colebrook', 'prospect', 'glastonbury', 'hampton', 'sprague',
       'canterbury', 'new london', 'bethel', 'morris', 'salisbury',
       'pomfret', 'north branford', 'east haven', 'kent', 'ansonia',
       'killingworth', 'chaplin', 'plainville', 'bloomfield', 'canton',
       'canaan', 'south windsor', 'washighton', 'hartland', 'eastford',
       'east hampton', 'new fairfield', 'warren', 'lyme', 'bristol',
       'plymouth', 'easton', 'suffield', 'e granby', 'simsbury',
       'portland', 'monroe', 'avon', 'no stonington', 'wolcott',
       'centerbrook', 'sprague - versailles', 'no canaan', 't165',
       'winsted', 'ridgefield', 'sprague - baltic', 'danielson',
       'ivoryton', 'wilton', 'storrs', 'e hartford', 'e lyme'],
      dtype=object)

In [66]:
df["officer_id"].unique()[df["officer_id"].value_counts()>50]
officer_list = np.array([55887736, 770626977, 1000002961, 1000002376, 70583331, 1000002714,
       1000003026, 1000002219, 298463910, 1000002220, 1000003191,
       1000003177, 1000002710, 1000003167, 653242422, 1000001884,
       277048371, 1000003181, 1000003235, 1000001983, 1000002599,
       1000003171, 1000003221, 1000002238, 1000002587, 1000002944,
       1000003198, 742774399, 1000002946, 540042328, 614680420,
       1000002270, 483102647, 104759822, 159668340, 1000001956, 642677890,
       1000002353, 469479530, 1000002703, 1000002613, 934977327,
       1000003163, 1000002591, 747377440, 1000002244, 1000002259,
       953549550, 1000002256, 853847642, 702562013, 1000002569,
       1000002708, 710252242, 116991428, 1000001923, 1000003230,
       322409049, 1000002750, 308565279, 1000003205, 637071474,
       1000002702, 1000002959, 1000002208, 1000002479, 1000003211,
       1000001939, 19018, 706750556, 1000002747, 1000003213, 520878322,
       1000002608, 1000003215, 1000002385, 927601465, 31240166, 331752178,
       1000003196, 1000002377, 1000002761, 18907, 1000001893, 1000003021,
       607163196, 1000002546, 257011101, 829157681, 26013717, 1000002235,
       230931989, 668908943, 722065405, 1000003194, 533883416, 944661814,
       915131263, 1000002728, 824655496, 908158929, 1000002362,
       1000003174, 1000002748, 599200807, 1000002240, 1000003017,
       1000002727, 1000002875, 1000003162, 1000003229, 1000001940,
       1000003363, 726828835, 872476022, 1000001996, 190163716,
       1000003156, 1000002617, 1000002598, 1000002940, 1000002575,
       1000003182, 265069536, 71903124, 1000002939, 1000003387,
       1000002349, 960967070, 424481709, 1000003031, 1000002952,
       713140385, 1000002380, 413162233, 668836964, 1000003186, 124039771,
       1000002576, 424133385, 1000002760, 1000003203, 13468937, 329593147,
       1000003055, 1000003029, 293708152, 567049933, 1000003160, 78218753,
       785469339, 1000002237, 41354688, 1000003384, 1000003022,
       1000003272, 940662250, 335699444, 279965403, 1000003353, 725428335,
       807433607, 1000002740, 418117108, 1000001914, 1000001949,
       1000003195, 826689753, 1000002726, 1000002602, 1000003223,
       684190569, 1000002954, 106840086, 1000003225, 1000002634,
       439075424, 1000002623, 1000002243, 19172, 1000002960, 1000003392,
       1000002371, 1000002271, 1000002097, 1000002721, 381521852,
       1000002942, 418720717, 1000003168, 253215760, 89095882, 1000003399,
       1000003208, 29992679, 498167384, 495322141, 861164535, 1000002949,
       1000002217, 1000002590, 1000003034, 1000002758, 1000001920,
       1000003175, 1000003155, 1000003189, 1000002743, 1000003207,
       1000002957, 1000001918, 790642042, 506911367, 18893, 961269420,
       1000003197, 1000002955, 1000001903, 983655904, 899734338, 46852777,
       565408419, 1000003154, 415391716, 1000002732, 1000003092,
       1000001917, 562015716, 1000002762, 708150264, 1000003037,
       1000003377, 1000002733, 19012, 577714579, 324857301, 671622127,
       488417172, 167438590, 1000002730, 1000002763, 1000002265,
       697576848, 19209, 1000001922, 106659415, 1000002855, 1000002752,
       1000003176, 941674240, 939084757, 569478224, 88120, 512066154,
       1000002549, 1000003166, 1000002618, 363177935, 23041953,
       1000001885, 1000002604, 1000002363, 634601902, 1000002906,
       1000002606, 1000002707, 607702361, 480852965, 1000003310,
       113658284, 1000003124, 1000002578, 480991900, 201145055, 178825628,
       241315469, 1000001899, 11486393, 94016, 627195004, 381140169,
       1000002715, 1000002233, 726303957, 88185785, 1000003200, 875724425,
       1000002593, 1000002709, 1000002586, 448051592, 987312802,
       1000003234, 1000003192, 907060051, 462749467, 258421805,
       1000003164, 848339269, 641939147, 938887034, 1000002624,
       1000001952, 1000003032, 970826817, 1000001935, 1000002724,
       1000003397, 1000002207, 61208768, 1000002204, 1000003169,
       1000001927, 349680451, 1000003183, 1000002869, 120121335,
       556481407, 336159473, 1000002384, 257227481, 759517227, 1000001941,
       612222124, 281119621, 923295155, 1000002723, 830814942, 1000003227,
       910711431, 1000003218, 639567735, 389731444, 71080948, 1000003024,
       764772746, 1000002221, 1000001998, 735104535, 1000002565,
       1000002592, 1000002741, 288030420, 95565661, 1000003352,
       1000002950, 1000002369, 849715205, 1000003356, 1000001948,
       1000002625, 23609079, 1000003368, 240586212, 445842676, 10450,
       351062524, 447203558, 1000002895, 1000002705, 827637066,
       1000002600, 18936, 195480226, 680984616, 1000002370, 1000003226],
      dtype=object)

In [67]:
%%time
for list in location_list:
    df["location_"+list] = np.zeros(df.contraband_found.count())
    df.loc[df["location_raw"]==list,"location_"+list]=1

CPU times: user 1min 4s, sys: 5.28 s, total: 1min 9s
Wall time: 8.78 s


In [33]:
%%time
for list in officer_list:
    df["officer_"+str(list)] = np.zeros(df.contraband_found.count())
    df.loc[df["location_raw"]==str(list),"officer_"+str(list)]=1

CPU times: user 4min 3s, sys: 53.3 s, total: 4min 56s
Wall time: 1min 27s


In [68]:
df["location_I84"] = 0
df.loc[(df["fine_grained_location"].str.contains("84")),"location_I84"] = 1

df["location_I91"] = 0
df.loc[(df["fine_grained_location"].str.contains("91")) & ~(df["fine_grained_location"].str.contains("T091")),"location_I91"] = 1

df["location_I95"] = 0
df.loc[(df["fine_grained_location"].str.contains("95")) & ~(df["fine_grained_location"].str.contains("T095")),"location_I95"] = 1

df["location_I395"] = 0
df.loc[(df["fine_grained_location"].str.contains("395")) & ~(df["fine_grained_location"].str.contains("T395")),"location_I395"] = 1

In [69]:
df = df.drop(["fine_grained_location","location_raw","officer_id"],axis=1)

In [70]:
df.to_csv('test_clean_06131430.csv',sep=',',index=False)